In [6]:
import comm as cmm
import codecs

def fmtStrQuote(v):
    return str(v) if(type(v)==int) else f"'{str(v)}'"

def fmtOrderBy(v):
	return "" if(v=="") else f"""\t\t\t\tCASE WHEN @SortColumn = '{v}' AND @SortOrder = 'ASC' THEN f.CreateDT END ASC,\n\t\t\t\tCASE WHEN @SortColumn = '{v}' AND @SortOrder = 'DESC' THEN f.CreateDT END DESC,"""

def fmtQueryWhere(v):
    return "" if(v=="") else f""" f.{v} LIKE CASE WHEN @{v} = '' THEN f.{v} ELSE '%' + @{v} + '%' END\n\t\t\t\tAND"""

def fmtUpdateSet(v):
    return "" if(v=="") else f",{v} = @{v}"

def fmtInsertCols(v):
    return "" if(v=="") else f",[{v}]"

def fmtInsertVals(v):
    return "" if(v=="") else f",@{v}"

def GenCreateTable(df):
    tbNames = df["TableName"].unique()
    for tb in tbNames:
        colsStr, ColExtFormats="",""
        dfs = df[df["TableName"]==tb]
      
        for e in dfs["colComp"]:
            colsStr += e +"\n"

        for e in dfs["colExt"]:
            ColExtFormats += e +"\n"

        tmpFile = codecs.open("template/createTable.sql","r").read().format(tb=tb, colsStr=colsStr, ColExtFormats=ColExtFormats)
        f = codecs.open(f"tables/tb{tb}.sql", "w","utf-8")
        f.write(tmpFile)
        f.close()


In [36]:
import pandas as pd
import numpy as np
import pyodbc
import comm as cmm
import time
import codecs

df = pd.read_excel("tableSourceDef.xlsx", sheet_name="admin_columns",converters={'maxLen':str }).replace(np.nan, '')
df_sp = pd.read_excel("tableSourceDef.xlsx", sheet_name="SP_config",converters={'maxLen':str }).replace(np.nan, '')
df["colComp"] = "\t"+df["ColumnName"].apply(cmm.fmtSquare) + (df["Type"]+df["maxLen"].apply(cmm.fmtParentheses)).apply(cmm.fmtSquare) + df["autoGen"]+ df["isNull"].apply(cmm.fmtNull)+","
df["colExt"] = "EXEC sys.sp_addextendedproperty @name=N'MS_Description', @value=N'" + df["ColumnDscr"]+ "' ,@level0type=N'SCHEMA',@level0name=N'" +df["Schema"]+"', @level1type=N'TABLE',@level1name=N'tb"+ df["TableName"]+"', @level2type=N'COLUMN',@level2name=N'"+df["ColumnName"]+"'\nGO"
   
GenCreateTable(df)  ## 自動產生Tables

cnxn = pyodbc.connect(cmm.connMSSQL("newcti"))
query="select SN,tableName,col,fCol,colName,DataType,cmmt_select,cmmt_InputParas,cmmt_Declares,cmmt_SetValue,cmmt_Exec,fColOr,[sp_Update]  from [dbo].[myTbColumnDscr];"
dfc = pd.read_sql_query(query, cnxn).set_index("SN")
tbs = df_sp["tableName"].unique()


def GenSP(tb, df, dfc, keyCol, tmpFileTag):
    tbDscr = df_sp[df_sp["tableName"] == tb]["tbDscr"].values[0]

    df_IsExist = df[~(df[keyCol]=="")]                                      #主軸欄位:KeyColumnName in Excel.
    dfc_IsExist = dfc[dfc["col"].isin(df_IsExist["ColumnName"].tolist())]   #衍生出的另一個dataframe
    target = df_IsExist.merge(dfc_IsExist, left_on="ColumnName", right_on="col")
    target["setValued"] = target["cmmt_SetValue"] + target["HttpGetSetValue"].apply(fmtStrQuote)

    orderByList = df[df["orderBy"]=="V"]["ColumnName"].tolist()             #找出orderBy欄位有哪些欄位
    df_OrderBy = df[df["ColumnName"].isin(orderByList)]

    queryWhereList = df[df["queryWhere"]=="V"]["ColumnName"].tolist()       #擷取queryWhere有哪些欄位
    df_QueryWhere = df[df["ColumnName"].isin(queryWhereList)]

    excludeForUpdate = ["SeqNo","Creator","CreateDT","UpdateDT","Updator"]
    updateSetList = df[df["ColumnName"].isin(excludeForUpdate)]["ColumnName"].tolist()    #擷取updateSet有哪些欄位，同時排除SeqNo
    df_UpdateSet = df[~df["ColumnName"].isin(updateSetList)]
    target["insertSetVal"] = "SET @" + target["ColumnName"] + " = " + target["httpInsert"].apply(fmtStrQuote)

    pt_input, pt_Declare, pt_SetValue,pt_select, pt_fCol,pt_Exec,pt_fColOr,pt_orderBy,pt_queryWhere,pt_UpdateSet,pt_insertCols,pt_insertSetVal,pt_insertVals = "","","","","","","","","","","","",""

    target2 = target[target["col"]!="SeqNo"]  
    pt_select = "\n".join(target["cmmt_select"])
    pt_input = "\n\t".join(target["cmmt_InputParas"])
    pt_Declare = "\n\t".join(target["cmmt_Declares"])[1:]
    pt_SetValue = "\n\t".join(target["setValued"].tolist())
    pt_fCol = "\n\t\t\t".join(target["fCol"])
    pt_Exec = "\n\t".join(target["cmmt_Exec"])[1:]
    pt_fColOr = " OR\n\t\t\t\t".join(target2["fColOr"])
    pt_orderBy ="\n".join(df_OrderBy["ColumnName"].apply(fmtOrderBy).tolist())[:-1]
    pt_queryWhere ="".join(df_QueryWhere["ColumnName"].apply(fmtQueryWhere).tolist())[:-8]
    pt_UpdateSet ="\n\t\t\t".join(df_UpdateSet["ColumnName"].apply(fmtUpdateSet).tolist())[1:]
    pt_insertCols = "\n\t\t\t".join(df_UpdateSet["ColumnName"].apply(fmtInsertCols).tolist())[1:]
    pt_insertSetVal = "\n\t".join(target["insertSetVal"].tolist())
    pt_insertVals = "\n\t\t\t".join(df_UpdateSet["ColumnName"].apply(fmtInsertVals).tolist())[1:]

    tmpQueryFile = codecs.open(f"template/agdSP.{tmpFileTag}.sql",mode="r",encoding="utf-8",errors='strict', buffering=-1).read().format(tb=tb, tbDscr=tbDscr, pt_DateTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), pt_select = pt_select, pt_input = pt_input,pt_Declare= pt_Declare, pt_SetValue = pt_SetValue, pt_Exec = pt_Exec, pt_fCol=pt_fCol[1:], pt_fColOr = pt_fColOr, pt_orderBy = pt_orderBy, pt_queryWhere = pt_queryWhere, pt_UpdateSet = pt_UpdateSet, pt_insertCols = pt_insertCols, pt_insertVals = pt_insertVals,pt_insertSetVal = pt_insertSetVal)
    f = codecs.open(f"agdSP/agdSP.usp{tb}{tmpFileTag}.sql", mode="w", encoding="utf-8", errors="strict")
    f.write(tmpQueryFile)
    f.close()

for tb in tbs:
    _df, _dfc = df[df["TableName"] == tb], dfc[dfc["tableName"] == tb]

    GenSP(tb, _df, _dfc, "isExist","Query")
    GenSP(tb, _df, _dfc, "isExist","Exists")
    GenSP(tb, _df, _dfc, "isExist","Get")
    GenSP(tb, _df, _dfc, "isExist","Delete")
    GenSP(tb, _df, _dfc, "httpInsert","Insert")
    GenSP(tb, _df, _dfc, "httpUpdate","Update")
    

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
